In [ ]:
# 1) (Optional) Mount Google Drive to persist checkpoints
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception as e:
    print('Not running in Colab or drive mount failed:', e)

In [ ]:
# 2) Clone the repository (or pull if already present)
import os, subprocess, sys
if not os.path.exists('270FT'):
    subprocess.check_call([sys.executable, '-m', 'git', 'clone', 'https://github.com/oleeveeuh/270FT.git'])
else:
    print('Repository already exists; fetching latest changes')
    try:
        subprocess.check_call(['git', '-C', '270FT', 'pull'])
    except Exception as e:
        print('git pull failed:', e)
# Change working directory to repo root
os.chdir('270FT')
print('CWD:', os.getcwd())

In [ ]:
# 3) Install required Python packages (may take a few minutes).
# Install core packages first, then attempt bitsandbytes which may need a matching CUDA runtime.
!pip install -q transformers datasets peft evaluate pyyaml huggingface_hub wandb
# Try to install bitsandbytes; if it fails, you can retry with a wheel matching the runtime's CUDA
try:
    get_ipython().system('pip install -q bitsandbytes')
    print('bitsandbytes installed')
except Exception as e:
    print('bitsandbytes install failed (you may still proceed if using a different runtime):', e)
# Show GPU info if available
!nvidia-smi || true

If `bitsandbytes` or other installs fail due to CUDA mismatches, try switching the Colab runtime GPU type (or use a notebook with a supported CUDA version). You can also skip installing `bitsandbytes` if you only want CPU runs, but the full QLoRA workflow requires a CUDA GPU.

In [ ]:
# 4) (Optional) Preprocess raw data into processed JSONL files.
# This will create/update data/processed/train.jsonl, validation.jsonl, and test.jsonl
!python preprocess/load_and_prepare.py --raw_dir data/raw --processed_dir data/processed --validation_split 0.15 --test_split 0.15

In [ ]:
# 5) (Optional) Inspect processed files sizes and line counts
!ls -lh data/processed || true
!wc -l data/processed/*.jsonl || true

In [ ]:
# 6) Provide Hugging Face token (if required).
# Use this cell to securely input a token if you need to access gated models.
from getpass import getpass
token = getpass('Hugging Face token (leave blank if not needed): ')
import os
if token:
    os.environ['HF_TOKEN'] = token
    # Also login via huggingface-cli for convenience
    try:
        get_ipython().system('huggingface-cli login --token "$HF_TOKEN"')
    except Exception as e:
        print('Automatic huggingface-cli login failed; token stored in HF_TOKEN')
    print('HF_TOKEN set in environment')
else:
    print('No token provided; proceeding without HF token')

In [ ]:
# 6b) Disable W&B interactive logging to avoid login prompts during automated runs
import os
os.environ['WANDB_MODE'] = 'offline'
print('W&B offline mode set')

Now run the full training script below. The script `training/train_dual_lora.py` will look for `data/processed/train.jsonl`, `validation.jsonl`, and `test.jsonl` in `data/processed` and will use the models listed in `configs/training_config.yaml`.

**Note about test data**: The test set is reserved for human-in-the-loop evaluation after training. During training, only the validation set is used for evaluation metrics. The training will log:
- **Training loss** every 10 steps
- **Validation loss** at the end of each epoch
- Metrics to W&B (if enabled) or console output

In [ ]:
# 7) Run the training script (this will start QLoRA fine-tuning).
# Note: training will log to W&B if enabled in the config; we set W&B to offline above to avoid interactive prompts.
# Use unbuffered output so logs stream in Colab
!python -u training/train_dual_lora.py

**Training Monitoring:**
The training script logs losses and metrics in multiple ways:
1. **Console output**: Training loss logged every 10 steps, validation metrics at each epoch
2. **W&B (if enabled)**: Full training/validation curves, model checkpoints
3. **Checkpoints**: Saved every 500 steps to the output directory (keeping last 3)

**Troubleshooting tips:**
- If you see `FileNotFoundError` complaining about missing `test.jsonl`, re-run the preprocessing cell or ensure `data/processed/test.jsonl` exists.
- If you get `bitsandbytes` import errors, try installing a different `bitsandbytes` wheel that matches the Colab CUDA runtime or switch runtime.
- If training runs out of VRAM, reduce `batch_size` in `configs/training_config.yaml` or switch to a larger GPU.
- If you prefer to persist checkpoints to Google Drive, create a folder in Drive and update `configs/training_config.yaml` output paths to point inside `/content/drive/MyDrive/...`.

**Next steps after training:**
- Run inference on test questions using the trained adapter
- Use human-in-the-loop review to evaluate the quality of generated solutions
- The test set at `data/processed/test.jsonl` contains questions without solutions (intentional for unbiased evaluation)